In [14]:
from sdm.utils import set_project_wd
set_project_wd()

Current Working Directory: /Users/matthewwhittle/Data Science/shefflied-bats


In [15]:
import geopandas as gpd
import pandas as pd

In [16]:
boundary = gpd.read_file("data/processed/boundary.geojson")
bats = gpd.read_parquet("data/processed/sybg-bats.parquet")

In [17]:
bats.head()

,grid_reference,species_raw,activity_type,source_data,date,latin_name,common_name,genus,x,y,accuracy,geometry,grid_square_geom
0,NZ115084,Common pipistrelle,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1274054400000,""Gr...",2010-05-17 00:00:00,Pipistrellus pipistrellus,Common Pipistrelle,Pipistrellus,411550.0,508450.0,100.0,POINT (411550.000 508450.000),"POLYGON ((411600.000 508400.000, 411600.000 50..."
1,NZ14640021,Pipistrellus sp.,Unknown,"{""Recorder"":""Giles Manners"",""Date"":14043456000...",2014-07-03 00:00:00,Pipistrellus sp.,Unidentified Pipistrelle,Pipistrellus,414645.0,500215.0,10.0,POINT (414645.000 500215.000),"POLYGON ((414650.000 500210.000, 414650.000 50..."
2,NZ20291106,Soprano pipistrelle,Unknown,"{""Recorder"":""Natural England Volunteer Bat Roo...",2013-11-28 00:00:00,Pipistrellus pygmaeus,Soprano Pipistrelle,Pipistrellus,420295.0,511065.0,10.0,POINT (420295.000 511065.000),"POLYGON ((420300.000 511060.000, 420300.000 51..."
3,NZ170014,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1283212800000,""Gr...",2010-08-31 00:00:00,Unknown,Unidentified Bat,Unknown,417050.0,501450.0,100.0,POINT (417050.000 501450.000),"POLYGON ((417100.000 501400.000, 417100.000 50..."
4,NZ185116,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1242777600000,""Gr...",2009-05-20 00:00:00,Unknown,Unidentified Bat,Unknown,418550.0,511650.0,100.0,POINT (418550.000 511650.000),"POLYGON ((418600.000 511600.000, 418600.000 51..."


In [22]:
bats["date"] = pd.to_datetime(bats["date"])
# Generate a unique ID based upon the date, grid reference and species 
def generate_id(row) -> str:
    # Check the date is present
    if pd.isnull(row.date):
        date = "unknown"
    else:
        date = row.date.strftime('%Y-%m-%d')
    return f"{date}_{row.grid_reference}_{row.latin_name}_{row.activity_type}"

bats["id"] = bats.apply(generate_id, axis=1)

bats.head()

,grid_reference,species_raw,activity_type,source_data,date,latin_name,common_name,genus,x,y,accuracy,geometry,grid_square_geom,id
0,NZ115084,Common pipistrelle,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1274054400000,""Gr...",2010-05-17,Pipistrellus pipistrellus,Common Pipistrelle,Pipistrellus,411550.0,508450.0,100.0,POINT (411550.000 508450.000),"POLYGON ((411600.000 508400.000, 411600.000 50...",2010-05-17_NZ115084_Pipistrellus pipistrellus_...
1,NZ14640021,Pipistrellus sp.,Unknown,"{""Recorder"":""Giles Manners"",""Date"":14043456000...",2014-07-03,Pipistrellus sp.,Unidentified Pipistrelle,Pipistrellus,414645.0,500215.0,10.0,POINT (414645.000 500215.000),"POLYGON ((414650.000 500210.000, 414650.000 50...",2014-07-03_NZ14640021_Pipistrellus sp._Unknown
2,NZ20291106,Soprano pipistrelle,Unknown,"{""Recorder"":""Natural England Volunteer Bat Roo...",2013-11-28,Pipistrellus pygmaeus,Soprano Pipistrelle,Pipistrellus,420295.0,511065.0,10.0,POINT (420295.000 511065.000),"POLYGON ((420300.000 511060.000, 420300.000 51...",2013-11-28_NZ20291106_Pipistrellus pygmaeus_Un...
3,NZ170014,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1283212800000,""Gr...",2010-08-31,Unknown,Unidentified Bat,Unknown,417050.0,501450.0,100.0,POINT (417050.000 501450.000),"POLYGON ((417100.000 501400.000, 417100.000 50...",2010-08-31_NZ170014_Unknown_Roost
4,NZ185116,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1242777600000,""Gr...",2009-05-20,Unknown,Unidentified Bat,Unknown,418550.0,511650.0,100.0,POINT (418550.000 511650.000),"POLYGON ((418600.000 511600.000, 418600.000 51...",2009-05-20_NZ185116_Unknown_Roost


In [31]:
bats_unique = bats.drop_duplicates(subset=["id"])

In [38]:
# drop bats after 2000
bats_unique = bats_unique[bats_unique.date < "2000-01-01"]

In [36]:
bats_unique.drop(columns=["grid_square_geom"], inplace=True)

/var/folders/vm/9ws15vws62zgnr64rdr_ls5r0000gn/T/ipykernel_45914/1730682156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bats_unique.drop(columns=["grid_square_geom"], inplace=True)


In [44]:
for species in bats_unique.latin_name.unique():
    for activity in bats_unique.activity_type.unique():
        bats_subset = bats_unique[(bats_unique.latin_name == species) & (bats_unique.activity_type == activity)]
        if len(bats_subset) == 0:
            continue
        bats_subset.to_file("data/raw/bats-pre-2000.gpkg", layer=f"{species}-{activity}", driver="GPKG")